# **Data Analysis Assignment 1: Analyzing your own EEG activity from a visual oddball ERP experiment**

**Dataset**: CSV file containing your own Muse EEG activity recorded during a stimulus-evoked ERP experiment (Visual Oddball)

### **Learning goals**

By the end of this notebook, you will be able to 

1. Import and sanity-check real EEG data collected by you.

2. Create event-locked **epochs** around stimulus onset.

3. Compute and plot **average ERPs** for standard vs. deviant stimuli at each electrode.

4. Visualize uncertainty with **SEM shaded error** around ERP traces.

5. Quantify ERP magnitude by finding the maximum value within a **time-window average** (200-400 ms) and summarize with bar plots + SEM errorbars.

### **1) Setup: imports, load data file**

In [ ]:
# Let's start by doing our standard imports first! 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# Optional import 

%matplotlib inline

**File Loading Instructions**

1. Put your CSV file in the same folder as this notebook, or paste the full file path below.

2. If you get an error, check the filename and extension carefully. (On Google Colab, make sure to check where the file is located as well!)

In [ ]:
csv_path = "" # <-- Put your dataset file name here! (Make sure it contains the .csv at the end as well)

df = pd.read_csv(csv_path)

df.head()

> **After running the code above, answer QUESTION 1 in your Assignment 1 document.**

### **2) Inspect the data table (sanity checks)**

**Inspecting the raw table** 

Before doing any signal processing, we need to confirm: 

- Which columns correspond to EEG channels (the 4 electrodes) 

- Where the **event markers** live (the information that gives us **stimulus onset times** and **condition labels**)

In [ ]:
df.head() 

df.shape, df.columns

> **After running the code above, answer QUESTION 2 in your Assignment 1 document.**

### **3) Extract relevant columns from the dataset** 

**Columns we will extract:** 

- Time: <span style="color:green">Timestamp (ms)</span> (our dataset contains absolute time in ms. We will convert to seconds relative to the first sample).

- Condition / event: <span style="color:green">Marker</span> (10 = standard, 20 = deviant).

- Channels: <span style="color:green">ch0_4ms ... ch3_4ms</span>

In [ ]:
# Sample rate in Hz 
fs = 250 

# The dataset expresses time in absolute ms. Let's convert to seconds relative from the first sample (i.e., the start of our recording).
time_ms = df["Timestamp (ms)"].to_numpy()
time_s = (time_ms - time_ms[0]) / 1000.0

marker = df["Marker"].to_numpy()

channel_cols = ["ch0_4ms", "ch1_4ms", "ch2_4ms", "ch3_4ms"]
data = df[channel_cols].to_numpy() 

# The shape of the data should be (n_samples, 4)
time_s[:5], data.shape

> **After running the code above, answer QUESTION 3 in your Assignment 1 document.**

### **4) Detect event onsets**

Time to detect when exactly our two types of stimuli (standards and deviants) occurred. We define event onsets as rows where the <span style="color:green">Marker</span> column = 10 for standard stimuli, or 20 for deviant stimuli. 

In [ ]:
epoch_start = int(round(fs*0.1))
epoch_end = int(round(fs*0.5))

# event when we go from 0 to 10 or 0 to 20
event_mask = (marker > 0) & (np.roll(marker, 1) == 0)

# boundary constraints 
valid = np.zeros_like(marker, dtype=bool) 
valid[epoch_start : len(data) - epoch_end] = True

event_ix = np.where(event_mask & valid)[0]  # indices of event samples

# Split by condition
std_ix = event_ix[marker[event_ix] == 10]
dev_ix = event_ix[marker[event_ix] == 20]

# output the number of standard and deviant trials found in the data:
len(std_ix), len(dev_ix)

> **After running the code above, answer QUESTION 4 in your Assignment 1 document.**

### **5) Epoch the data**

Next, we want to epoch the continuous data from [-0.1 to 0.5] seconds. At a sampling frequency of 250 Hz:

- 0.1s = 250 * 0.1 = 25 samples

- 0.5s = 125 samples

Therefore the total epoch length in samples = 25 + 125 + 1 = 151 samples (including time = 0).

In the code below, we will: 

1. In the first code cell, we'll build a variable called <span style="color:green">times</span> that is a long list of numbers (a vector) containing the times of each sample in our epoch

2. In the second cell, we'll slice windows of EEG data around each event index to create epochs of brain activity and then store the epoched data as a 3-D variable, with the dimensions **trials** x **time** x **channels**.

In [ ]:
tmin, tmax = -0.1, 0.5
smin = int(round(tmin * fs)) # negative
smax = int(round(tmax * fs)) # positive
sample_offsets = np.arange(smin, smax + 1) # e.g., -25 ... +125
times = sample_offsets / fs

times[0], times[-1], len(times)

In [ ]:
# params: 
# - data: (n_samples, n_channels)
# - returns: epochs (n_trials, n_times, n_channels) and kept event indices
def make_epochs(data, event_indices, epoch_start, epoch_end):
    epochs = [] 
    for t in event_indices: 
        if (t - epoch_start >= 0) and (t + epoch_end + 1 <= len(data)): 
            epochs.append(data[t - epoch_start : t + epoch_end + 1, :])
    return np.array(epochs)

epochs_std = make_epochs(data, std_ix, epoch_start, epoch_end)
epochs_dev = make_epochs(data, dev_ix, epoch_start, epoch_end)

epochs_std.shape, epochs_dev.shape

> **After running the code above, answer QUESTION 5 in the Assignment 1 document.**

### **6) Plot average ERPs**

We will now compute the mean across trials for each stimulus type (standard vs. deviant) and for each electrode.

In [ ]:
mean_std = epochs_std.mean(axis=0)  # (times, ch)
mean_dev = epochs_dev.mean(axis=0)

for ci, ch in enumerate(channel_cols):
    plt.figure()
    plt.plot(times, mean_std[:, ci], label="Standard (10)")
    plt.plot(times, mean_dev[:, ci], label="Deviant (20)")
    plt.axvline(0, linestyle="--")
    plt.title(f"Average ERP - {ch}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (a.u.)")
    plt.legend()
    plt.show()


> **After running the code above, COPY AND PASTE the figure into QUESTION 6 in your Assignment 1 document and answer the subsequent questions.**

### **7) Add SEM shaded error around traces** 

The standard error of the mean (SEM) at each time-point shows uncertainty in the estimated mean ERP across trials. Let’s add this to our plot of ERPs.

In [ ]:
# params: 
# - epochs: (trials, times, ch)
def sem_over_trials(epochs): 
    sd = epochs.std(axis=0, ddof=1)
    n = epochs.shape[0]
    
    return sd/np.sqrt(n)

sem_std = sem_over_trials(epochs_std)
sem_dev = sem_over_trials(epochs_dev)

for ci, ch in enumerate(channel_cols):
    plt.figure()
    for m, s, label in [
        (mean_std[:, ci], sem_std[:, ci], "Standard (10)"),
        (mean_dev[:, ci], sem_dev[:, ci], "Deviant (20)")
    ]:
        plt.plot(times, m, label=label)
        plt.fill_between(times, m - s, m + s, alpha = 0.25)
        
    plt.axvline(0, linestyle="--")
    plt.title(f"ERP with SEM - {ch}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (a.u.)")
    plt.legend()
    plt.show()

> **After running the code above, COPY AND PASTE the figure into QUESTION 7 in your Assignment 1 document and answer the questions.**

### **8) Find maximum response within a time window of 200-400m** 

To quantify the differences you’re seeing in your ERPs of standard vs. deviant stimuli, we will compute a single number per ERP: max amplitude between 0.2–0.4s. We’ll then plot a bar graph with error bars displaying the SEM. 

In [ ]:
win_mask = (times >= 0.200) & (times <= 0.400)

max_mean_std = mean_std[win_mask, :].max(axis=0)
max_mean_dev = mean_dev[win_mask, :].max(axis=0)

for ci, ch in enumerate(channel_cols):
    print(f"Channel {ch}: Standard peak = {max_mean_std[ci]:.4f}; Deviant peak = {max_mean_dev[ci]:.4f}")

In [ ]:
x = np.arange(len(channel_cols))
width = 0.35

plt.figure()
plt.bar(x - width/2, max_mean_std, width, label="Standard (10)")
plt.bar(x + width/2, max_mean_dev, width, label="Deviant (20)")
plt.xticks(x, channel_cols)
plt.axhline(0, color="k", linewidth=0.8)
plt.title("Peak of mean ERP (200-400 ms)")
plt.ylabel("Amplitude (a.u.)")
plt.legend()
plt.show()

> **After running the code above, COPY AND PASTE the figure into QUESTION 8 in your Assignment 1 document and answer the questions.**

🎉 You’ve finished Assignment 1! Make sure to submit the **GOOGLE DOC** instead of this notebook.
